In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import re 
import json
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge

le = LabelEncoder()
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from scipy import stats



In [12]:
def preprocess(data):
    # Replace single quotes with double quotes in "random load mesures" column
    data["random load mesures"] = data["random load mesures"].str.replace("'",'"')
    data['building'] = data['building'].str.replace('Building_',"").astype(int)
    data['Town'] = le.fit_transform(data['Town'])
    data['Total Wall Area'] = data['Wall Rt'] * data['Height'] * data['Number of Floors']
    #data['Total Window Area'] = data['Windows Rt'] * data['Height'] * data['WWR'] * data['Number of Floors']
    data['Total Floor Area'] = data['Ground Floor Rt'] * data['Height']
    data['Total Roof Area'] = data['Roof Rt'] * data['Height']
    data['Total Internal Wall Area'] = data['Internal Wall Rt'] * data['Height'] * data['Number of Floors']
    data['Total Internal Floor Area'] = data['Internal Floor Rt'] * data['Height']
    data['Overall Energy Consumption'] = data['Operating Hours'] * data['EUI']

    
    pattern = r'\d+\.\d+'
    wwr = data['WWR'].apply(lambda x: max([float(m) for m in re.findall(pattern, x)]))
    data['WWR'] = wwr
    # assuming 'File' contains unique file names for each building shape, it can be removed
    
    
    features = data['random load mesures'].apply(lambda x: json.loads(x))
    cooling = features.apply(lambda x: float(x['Cooling'].replace(":C","")))
    lights = features.apply(lambda x: float(x['Lights'].replace(":C","")))
    data['Coolings'] = cooling
    data['Lights'] = lights
    
    data['Lighting Impact on Heat Load'] = data['Lights'] * data['Light Heat Gain']
    data['Energy Efficiency'] = data['EUI'] / data['Number of Floors']
    data['Lighting Heat Addition Rate'] = data['Light Heat Gain'] / data['Operating Hours']
    

    scaler = StandardScaler()
    data[['Cooling Setpoint','Coolings','Lights', 'EUI', 'Cooling COP', 'Operating Hours', 'WWR',      'Equipment Heat Gain', 'Internal Wall Rt', 'Internal Floor Rt', 'Infiltration','Ground Floor Rt', 'Number of Floors', 'Occupancy', 'Light Heat Gain', 'Windows Rt',      'Height', 'Heating COP', 'Heating Setpoint', 'Wall Rt', 'Start Time', 'windows g-value',      'Roof Rt', 'Boiler Efficiency', 'Internal Mass', 'Permeability', 'Total Floors Area',]] = scaler.fit_transform(data[['Cooling Setpoint','Coolings','Lights', 'EUI', 'Cooling COP', 'Operating Hours', 'WWR',                                                              'Equipment Heat Gain', 'Internal Wall Rt', 'Internal Floor Rt', 'Infiltration',                                                              'Ground Floor Rt', 'Number of Floors', 'Occupancy', 'Light Heat Gain', 'Windows Rt',                                                              'Height', 'Heating COP', 'Heating Setpoint', 'Wall Rt', 'Start Time', 'windows g-value',                                                              'Roof Rt', 'Boiler Efficiency', 'Internal Mass', 'Permeability', 'Total Floors Area']])
    
    interaction_features = ['Overall Energy Consumption', 'Lighting Impact on Heat Load', 'Energy Efficiency','Lighting Heat Addition Rate']
    for feature1 in interaction_features:
        for feature2 in interaction_features:
            if feature1 != feature2:
                interaction_feature_name = f'{feature1} x {feature2}'
                data[interaction_feature_name] = data[feature1] * data[feature2]

    # assuming we want to select the top 10 features with the highest F-test score
    selector = SelectKBest(f_regression, k=10)
    data = data.drop(['random load mesures', 'Permeability','File'], axis=1)
    #data = selector.fit_transform(data.drop('Operational Energy', axis=1), data['Operational Energy'])
    # assume your data is stored in a NumPy array called `data`
    for col in data.columns:
        if col == 'Operational Energy':
            continue  # skip target variable
        median = np.median(data[col])
        q1 = np.percentile(data[col], 25)
        q3 = np.percentile(data[col], 75)
        iqr = q3 - q1
        upper_bound = q3 + 1.5 * iqr
        lower_bound = q1 - 1.5 * iqr
        data.loc[data[col] > upper_bound, col] = median
        data.loc[data[col] < lower_bound, col] = median

    return data


In [13]:
train=pd.read_csv("/kaggle/input/traintrain/Train.csv")
test = pd.read_csv("/kaggle/input/testing/Test (1).csv")
csv_file = pd.DataFrame()
csv_file['submission id'] = test['building'] + '_Town_' + test['Town'].astype(str)

csv_file.head()

,submission id
0,Building_1_Town_1
1,Building_100_Town_1
2,Building_1000_Town_2
3,Building_10000_Town_0
4,Building_10005_Town_2


In [14]:
# Preprocess train and test data
train = preprocess(train)
test = preprocess(test)


In [15]:
from catboost import CatBoostRegressor


catboost_model = CatBoostRegressor()

X = train.drop(['Operational Energy'], axis=1)
y = train['Operational Energy']
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize the list to store the mean squared errors (MSEs)
mse_list = []

In [16]:
from sklearn.linear_model import LinearRegression
for train_index, val_index in kf.split(X):
    
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # Scale the data using StandardScaler
    scaler = StandardScaler()
    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    X_val_fold_scaled = scaler.transform(X_val_fold)
    
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=10)
    X_train_fold_pca = pca.fit_transform(X_train_fold_scaled)
    X_val_fold_pca = pca.transform(X_val_fold_scaled)
    alpha = 0.1  # regularization strength
    model=LinearRegression()
    model = Ridge(alpha=alpha)
    # Train the model
    model.fit(X_train_fold_pca, y_train_fold)
    
    # Make predictions on the validation set and calculate MSE
    y_pred = model.predict(X_val_fold_pca)
    mse = np.mean((y_val_fold - y_pred)**2)
    
    # Append the MSE to the list
    mse_list.append(mse)
    
# Print the mean of the MSEs
print("Mean MSE:", np.mean(mse_list))

Mean MSE: 1616406272.178697


In [17]:
y_4b=scaler.fit_transform(test)
y_4b=pca.transform(y_4b)
y_4b = model.predict(y_4b)
y_4b

array([ 89313.39338672,  98883.8728327 , 100550.19671778, ...,
       187989.57381425,  49419.13424275, 199207.69700204])

In [18]:
importances = model.coef_

# Sort feature importances in descending order
sorted_importances = sorted(zip(importances, X.columns), reverse=True)

# Print the feature importances
for importance, feature in sorted_importances:
    print(f"{feature}: {importance}")


EUI: 14262.681143365577
Cooling Setpoint: 12831.472406418443
building: 8540.112074341316
Infiltration: 8114.643836539316
Equipment Heat Gain: 3099.6441584998656
Operating Hours: -387.8323671454449
Cooling COP: -1287.6778978670993
WWR: -1588.445944124592
Internal Wall Rt: -4535.907759753471
Internal Floor Rt: -4812.043440061067


In [19]:
#importance_scores = catboost_model.feature_importances_
feature_importances = dict(zip(X.columns, sorted_importances))
top_8_features = sorted(feature_importances, key=feature_importances.get, reverse=True)[:8]
from sklearn.linear_model import LinearRegression
# Select only the top 5 features
X = X[top_8_features]
test = test[top_8_features]

# Set up the KFold object
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize the list to store the mean squared errors (MSEs)
mse_list = []

# Loop over the folds
for train_index, val_index in kf.split(X):
    
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Scale the features
    scaler = StandardScaler()
    X_train_fold = scaler.fit_transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)
    
    # Reduce dimensionality using PCA
    pca = PCA(n_components=2)
    X_train_fold = pca.fit_transform(X_train_fold)
    X_val_fold = pca.transform(X_val_fold)

    # Train the model and predict on the validation set
    model=LinearRegression()
    model.fit(X_train_fold, y_train_fold)
    y_pred = model.predict(X_val_fold)

    # Compute the mean squared error
    mse = np.mean((y_val_fold - y_pred)**2)
    mse_list.append(mse)

# Compute the average mean squared error over the folds
avg_mse = np.min(mse_list)
print("Average MSE:", avg_mse)

Average MSE: 3917445253.5085816


In [20]:
avg_mse = np.max(mse_list)
print("Average MSE:", avg_mse)

Average MSE: 4001749311.664489


In [21]:
y_4bb=scaler.fit_transform(test)
y_4bb=pca.transform(y_4bb)
y_4bb = model.predict(y_4bb)
y_4bb

array([169336.68158218, 138645.20800702, 178984.1696174 , ...,
       196738.27499202, 142545.58650617, 209757.49988912])

In [23]:
#csv_file=pd.read_csv("/kaggle/input/simple/SampleSubmiss
    
csv_file['Operational Energy']=y_4bb.astype(int)

csv_file.to_csv("labidi1551.csv",index=False)
type(csv_file['Operational Energy'][0])

numpy.int64

In [24]:
csv_file.head()

,submission id,Operational Energy
0,Building_1_Town_1,169336
1,Building_100_Town_1,138645
2,Building_1000_Town_2,178984
3,Building_10000_Town_0,184816
4,Building_10005_Town_2,167148
